In [7]:
!pip install textblob

In [8]:
import textblob            #to import
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
import nltk
nltk.download('punkt')

In [114]:
import csv
import collections

In [91]:
f = open('Software_neg1_to_1_20.csv')
csv_f = csv.reader(f)
rows = []
for row in csv_f:
    if row:
        if float(row[0]) < 0:
            rows.append([row[1] , ",neg"])
        elif float(row[0]) > 0:
            rows.append([row[1], ",pos"])
rows
with open('train.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write multiple rows
    writer.writerows(rows)

In [92]:
with open('train.csv', 'r') as fp:
    cl = NaiveBayesClassifier(fp, format="csv")

In [117]:
zero = 0
f = open('Software_neg1_to_1_1000.csv')
csv_f = csv.reader(f)
TP = 0
TN = 0
FP = 0
FN = 0
FPValues = collections.defaultdict(int)
FNValues = collections.defaultdict(int)
for row in csv_f:
    if row:
        total += 1
        if total > 20:
            prob_dist = cl.prob_classify(row[1])
            res = cl.classify(row[1])
            round(prob_dist.prob("pos"), 2)
            if (float(row[0]) < 0 and res == ",neg"):
                TN += 1
            elif (float(row[0]) > 0 and res == ",pos"):
                TP += 1
            elif (float(row[0]) > 0): #Supposed to be positive but yielded negative
                FNValues[round(prob_dist.prob(",neg"), 2)] += 1
                FN += 1
            elif float(row[0]) < 0:
                FPValues[round(prob_dist.prob(",pos"), 2)] += 1
                FP += 1
            else:
                zero += 1

print(TP, TN, FP, FN, zero)

693 4 161 25 117


In [118]:
print(FPValues, FNValues)

defaultdict(<class 'int'>, {1.0: 156, 0.94: 1, 0.99: 2, 0.97: 1, 0.91: 1}) defaultdict(<class 'int'>, {1.0: 20, 0.93: 1, 0.5: 1, 0.98: 1, 0.69: 1, 0.64: 1})


In [96]:
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
FMeasure = (2 * Precision * Recall) / (Precision + Recall)
print("Precision: ", Precision)
print("Recall: ", Recall)
print("F1-Score: ", FMeasure)
print("Accuracy ", (TP + TN)/980)

Precision:  0.8104886769964244
Recall:  0.9645390070921985
F1-Score:  0.8808290155440416
Accuracy  0.6979591836734694
